<a href="https://colab.research.google.com/github/pekgnee/pix2pix-tensorflow-for-flowers/blob/master/pix2pix_tensorflow_for_flowers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download 102 Category Flower Dataset
http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html

In [1]:
# Code to read file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

    100% |████████████████████████████████| 993kB 17.0MB/s 


In [2]:
link = 'https://drive.google.com/open?id=1Gidqta8-KNYVl8vCaPG6vwEV_Tjd0tlz'
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('102flowers.tgz')

1Gidqta8-KNYVl8vCaPG6vwEV_Tjd0tlz


In [0]:
import tarfile
tf = tarfile.open("102flowers.tgz")
tf.extractall()

In [0]:
import os.path
if not os.path.exists('/content/png'):
  !mkdir png
  
input_dir = 'jpg'
output_dir = 'png'

## Use code from pix2pix paper
https://affinelayer.com/pixsrv/  
https://github.com/affinelayer/pix2pix-tensorflow

In [5]:
!git clone https://github.com/affinelayer/pix2pix-tensorflow.git

Cloning into 'pix2pix-tensorflow'...
remote: Enumerating objects: 261, done.
remote: Total 261 (delta 0), reused 0 (delta 0), pack-reused 261
Receiving objects: 100% (261/261), 13.33 MiB | 566.00 KiB/s, done.
Resolving deltas: 100% (103/103), done.


In [0]:
# UNZIP ZIP
# 390flowers.zip come from the intermediate saving results from 
# https://github.com/pekgnee/Holistically-nested-edge-detection-for-flowers
# This is a subset 390 images from incomplete run of 
# 8189 images in 102 Category Flower Dataset (takes too much time)
import zipfile

print ("Uncompressing zip file")
zip_ref = zipfile.ZipFile('390flowers.zip', 'r')
zip_ref.extractall('.')
zip_ref.close()

Uncompressing zip file


## Set up directories

In [0]:
import os.path

if not os.path.exists('/content/edges'):
  !mkdir /content/edges

if not os.path.exists('/content/resized'):
  !mkdir /content/resized
  
if not os.path.exists('/content/flowers_training'):
  !mkdir /content/flowers_training
  
if not os.path.exists('/content/flowers_training_output'):
  !mkdir /content/flowers_training_output

if not os.path.exists('/content/flowers_test_raw'):
  !mkdir /content/flowers_test_raw

if not os.path.exists('/content/flowers_test_edges'):
  !mkdir /content/flowers_test_edges

if not os.path.exists('/content/flowers_test_resized'):
  !mkdir /content/flowers_test_resized

if not os.path.exists('/content/flowers_test'):
  !mkdir /content/flowers_test
  
if not os.path.exists('/content/flowers_test_output'):
  !mkdir /content/flowers_test_output

## Environment setup for Edges
This takes about 15~20 min to set the environment up.

In [16]:
!apt install -y caffe-cuda

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  caffe-tools-cuda cython3 fonts-lyx javascript-common libblosc1
  libcaffe-cuda1 libcublas9.1 libcudart9.1 libcurand9.1 libjs-jquery
  libjs-jquery-ui python-matplotlib-data python-tables-data python3-bs4
  python3-caffe-cuda python3-chardet python3-cycler python3-dateutil
  python3-decorator python3-gflags python3-h5py python3-html5lib
  python3-ipython python3-ipython-genutils python3-leveldb python3-matplotlib
  python3-networkx python3-nose python3-numexpr python3-pandas
  python3-pandas-lib python3-pexpect python3-pickleshare python3-pkg-resources
  python3-prompt-toolkit python3-protobuf python3-ptyprocess python3-pygments
  python3-pyparsing python3-pywt python3-scipy python3-simplegeneric
  python3-six python3-skimage python3-skimage-lib python3-tables
  python3-tables-lib python3-traitlets python3-tz python3-wcwidth
  python3-web

In [9]:
CAFFE_ROOT='/opt/caffe'

!apt install -y --no-install-recommends \
  build-essential \
  cmake \
  libatlas-base-dev \
  libboost-all-dev \
  libgflags-dev \
  libgoogle-glog-dev \
  libhdf5-serial-dev \
  libleveldb-dev \
  liblmdb-dev \
  liboctave-dev \
  libopencv-dev \
  libprotobuf-dev \
  libsnappy-dev \
  octave \
  protobuf-compiler

!mkdir -p {CAFFE_ROOT}
%cd {CAFFE_ROOT}
!git clone https://github.com/s9xie/hed .
!git checkout 9e74dd710773d8d8a469ad905c76f4a7fa08f945
# https://github.com/s9xie/hed/pull/23
!sed -i "s|add_subdirectory(examples)||g" CMakeLists.txt
# https://github.com/s9xie/hed/issues/11
!sed -i "647s|//||" include/caffe/loss_layers.hpp
!sed -i "648s|//||" include/caffe/loss_layers.hpp
# keijiro: ad-hoc fix for compilation error
!sed -i "s|, 0.0|, Dtype(0.0)|" src/caffe/layers/contrastive_loss_layer.cpp

!mkdir {CAFFE_ROOT}/build
%cd {CAFFE_ROOT}/build
!cmake -DCPU_ONLY=1 {CAFFE_ROOT}
!make

%cd {CAFFE_ROOT}
!curl -O http://vcl.ucsd.edu/hed/hed_pretrained_bsds.caffemodel
  
%cd /root
!octave --eval "pkg install -forge image"
!echo "pkg load image;" >> .octaverc
!curl -O https://pdollar.github.io/toolbox/archive/piotr_toolbox.zip
!unzip -qq -o piotr_toolbox.zip
!octave --eval "addpath(genpath('/root/toolbox')); savepath;"
!echo "#include <stdlib.h>" > wrappers.hpp
!cat /root/toolbox/channels/private/wrappers.hpp >> wrappers.hpp
!mv wrappers.hpp /root/toolbox/channels/private/wrappers.hpp

!mkdir /root/mex
%cd /root/toolbox/channels/private
!mkoctfile --mex -DMATLAB_MEX_FILE -o /root/mex/convConst.mex convConst.cpp
!mkoctfile --mex -DMATLAB_MEX_FILE -o /root/mex/gradientMex.mex gradientMex.cpp
!mkoctfile --mex -DMATLAB_MEX_FILE -o /root/mex/imPadMex.mex imPadMex.cpp
!mkoctfile --mex -DMATLAB_MEX_FILE -o /root/mex/imResampleMex.mex imResampleMex.cpp
!mkoctfile --mex -DMATLAB_MEX_FILE -o /root/mex/rgbConvertMex.mex rgbConvertMex.cpp
!octave --eval "addpath('/root/mex'); savepath;"

%cd /root
!curl -O https://raw.githubusercontent.com/pdollar/edges/master/private/edgesNmsMex.cpp
!octave --eval "mex edgesNmsMex.cpp"
!mv edgesNmsMex.mex /root/mex/

%cd /content

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
cmake is already the newest version (3.10.2-1ubuntu2).
libatlas-base-dev is already the newest version (3.10.3-5).
libboost-all-dev is already the newest version (1.65.1.0ubuntu1).
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
libopencv-dev is already the newest version (3.2.0+dfsg-4ubuntu0.1).
The following additional packages will be installed:
  libamd2 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libfftw3-bin libfftw3-dev libfftw3-long3 libfftw3-quad3 libfftw3-single3
  libfltk-gl1.3 libfltk1.3 libgflags2.2 libglpk40 libgoogle-glog0v5
  libgraphicsmagick++-q16-12 libgraphicsmagick-q16-3 libleveldb1v5 liblmdb0
  libmetis5 liboctave4 libosmesa6 libportaudio2 libprotobuf-lite10
  libqrupdate1 libqscintilla2-qt5-13 libqscintilla2-qt5-l10n libqt5opengl5
  libsuitesparseconfig5 libtext-unidecode-pe

## Edge Detection

In [13]:
%cd pix2pix-tensorflow

[Errno 2] No such file or directory: 'pix2pix-tensorflow'
/content/pix2pix-tensorflow


In [0]:
!python tools/process.py \
  --input_dir '../jpg' \
  --output_dir '../edges' \
  --operation edges

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Instructions for updating:
Colocations handled automatically by placer.
skipping 0 files that already exist
processing 390 files
2019-03-03 10:33:05.258860: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-03-03 10:33:05.259246: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x219f1e0 executing computations on platform Host. Devices:
2019-03-03 10:33:05.259299: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-03 10:33:05.353886: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-03 10:33:05.354466: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x219eb00 executing computations on platform C

In [17]:
!python tools/process.py \
  --input_dir '../flowers_test_raw' \
  --output_dir '../flowers_test_edges' \
  --operation edges

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Instructions for updating:
Colocations handled automatically by placer.
skipping 0 files that already exist
processing 10 files
2019-03-04 08:32:31.733162: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-04 08:32:31.733492: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x17ab340 executing computations on platform Host. Devices:
2019-03-04 08:32:31.733549: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-04 08:32:31.834787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-04 08:32:31.835437: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x17aac60 executing computations on platform CU

## Resize Images

In [0]:
!python tools/process.py \
  --input_dir '../jpg' \
  --operation resize \
  --output_dir '../resized'

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Instructions for updating:
Colocations handled automatically by placer.
skipping 0 files that already exist
processing 390 files
2019-03-03 12:36:55.886435: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-03-03 12:36:55.886743: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x14e31e0 executing computations on platform Host. Devices:
2019-03-03 12:36:55.886780: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-03 12:36:55.989412: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-03 12:36:55.989954: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x14e2b00 executing computations on platform C

In [18]:
!python tools/process.py \
  --input_dir '../flowers_test_raw' \
  --operation resize \
  --output_dir '../flowers_test_resized'

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Instructions for updating:
Colocations handled automatically by placer.
skipping 0 files that already exist
processing 10 files
2019-03-04 08:38:38.309594: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-04 08:38:38.309892: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1faf340 executing computations on platform Host. Devices:
2019-03-04 08:38:38.309952: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-04 08:38:38.409580: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-04 08:38:38.410174: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x1faec60 executing computations on platform CU

## Creating image pairs from existing images

In [0]:
!python tools/process.py \
  --input_dir '../png' \
  --b_dir '../resized' \
  --operation combine \
  --output_dir '../flowers_training'

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Instructions for updating:
Colocations handled automatically by placer.
skipping 0 files that already exist
processing 390 files
2019-03-03 12:46:04.130047: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200000000 Hz
2019-03-03 12:46:04.130287: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x339f1e0 executing computations on platform Host. Devices:
2019-03-03 12:46:04.130347: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-03 12:46:04.224749: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-03 12:46:04.225280: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x339eb00 executing computations on platform C

In [19]:
!python tools/process.py \
  --input_dir '../flowers_test_edges' \
  --b_dir '../flowers_test_resized' \
  --operation combine \
  --output_dir '../flowers_test'

Instructions for updating:
This op will be removed after the deprecation date. Please switch to tf.sets.difference().
Instructions for updating:
Colocations handled automatically by placer.
skipping 0 files that already exist
processing 10 files
2019-03-04 08:41:27.578084: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2300000000 Hz
2019-03-04 08:41:27.578342: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x195d340 executing computations on platform Host. Devices:
2019-03-04 08:41:27.578379: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2019-03-04 08:41:27.672604: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2019-03-04 08:41:27.673212: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x195cc60 executing computations on platform CU

## Training

In [7]:
# train the model (this may take 1-8 hours depending on GPU, on CPU you will be waiting for a bit)
!python pix2pix.py \
  --mode train \
  --output_dir '../flowers_training_output' \
  --max_epochs 200 \
  --input_dir '../flowers_training' \
  --which_direction AtoB

aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = None
display_freq = 0
flip = True
gan_weight = 1.0
input_dir = ../flowers_training
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = 200
max_steps = None
mode = train
ndf = 64
ngf = 64
output_dir = ../flowers_training_output
output_filetype = png
progress_freq = 50
save_freq = 5000
scale_size = 286
seed = 162236791
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.

In [21]:
!python pix2pix.py \
  --mode test \
  --output_dir '../flowers_test_output' \
  --input_dir '../flowers_test' \
  --checkpoint '../flowers_training_output'

loaded lab_colorization = False
loaded ndf = 64
loaded ngf = 64
loaded which_direction = AtoB
aspect_ratio = 1.0
batch_size = 1
beta1 = 0.5
checkpoint = ../flowers_training_output
display_freq = 0
flip = False
gan_weight = 1.0
input_dir = ../flowers_test
l1_weight = 100.0
lab_colorization = False
lr = 0.0002
max_epochs = None
max_steps = None
mode = test
ndf = 64
ngf = 64
output_dir = ../flowers_test_output
output_filetype = png
progress_freq = 50
save_freq = 5000
scale_size = 256
seed = 1125720955
separable_conv = False
summary_freq = 100
trace_freq = 0
which_direction = AtoB
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf

In [0]:
!zip -r /content/pix2pix-tensorflow/facades_test/images.zip /content/pix2pix-tensorflow/facades_test/images

  adding: content/pix2pix-tensorflow/facades_test/images/ (stored 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/62-targets.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/79-outputs.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/23-inputs.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/16-targets.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/64-targets.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/36-targets.png (deflated 1%)
  adding: content/pix2pix-tensorflow/facades_test/images/60-inputs.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/53-targets.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/67-inputs.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/13-targets.png (deflated 0%)
  adding: content/pix2pix-tensorflow/facades_test/images/32-targets.png (